In [2]:
import pandas as pd
import numpy as np
import os
import sys

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

from textwrap import wrap
import humanize

In [166]:
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [4]:
folder = '/Users/ghost/PycharmProjects/Rusagro/Data'

# Справочник по номенклатуре SKU

Читаем из файла таблицу с Номенклатурой SKU и удаляем ненужные поля:

In [ ]:
file_name = 'Номенклатура.xlsx'
path = os.path.join(folder, file_name)
dtypes = {'Код товара': str}
SKU = pd.read_excel(path, header=0, dtype=dtypes)

drop = ['Категория + бренд', 'Категория расширенная']
SKU.drop(drop, axis=1, inplace=True)

SKU[:3]

# Справочник по клиентам (объединенный с региональной структурой)

Читаем данные из файла с Региональной структурой и удаляем ненужные поля.  
Нам необходима информация, которой нет в справочнике Клиенты:  
* Области РФ  
* Канал продаж  
* РФ\Экспорт  
  
"Техническая область" - ключевое поле, по которому остальные данные будут подгружаться в справочник по Клиентам:

In [ ]:
file_name = 'Региональная структура.xlsx'
path = os.path.join(folder, file_name)
geography = pd.read_excel(path, header=0)

drop = ['Склад отгрузки', 'Торговая сеть', 'Дивизион', 'Страна']
geography.drop(drop, axis=1, inplace=True)

geography['Техническая область'] = geography['Техническая область'].str.lower()

geography.head()

Читаем данные из файла со справочником по Клиентам, удаляем ненужные поля, которые дальше не будем использовать.  
Добавляем по полю "Техническая область" данные из Региональной структуры. Больше нам это поле не понадобится - удаляем:

In [ ]:
file_name = 'Клиенты.xlsx'
path = os.path.join(folder, file_name)
client = pd.read_excel(path, header=0)

drop = ['Адрес', 'Склад отгрузки', 'Статус проверки', 'Канал сбыта']

client.drop(drop, axis=1, inplace=True)
client['Техническая область'] = client['Техническая область'].str.lower()

client = client.merge(geography, on='Техническая область', how='left', validate='m:1')
client.drop(['Техническая область'], axis=1, inplace=True)

client.head()

# Факт

Читаем из файлов данные по факту и сразу добавляем в итоговую таблицу поля "Год", "Месяц" и "Неделя" для дальнейшего анализа:

In [8]:
dtypes = {'Код клиента': str, 'Код товара': str, 'Объем': float}

for i in range(4):
    file_name = "Факт " + str(i + 1) + '.xlsx'
    path = os.path.join(folder, file_name)
    
    if i == 0:
        fact = pd.read_excel(path, header=0, dtype=dtypes)
    else:
        fact = fact.append(pd.read_excel(path, header=0, dtype=dtypes), ignore_index=True)

In [9]:
fact['Месяц'] = fact['Дата'].dt.month
fact['Год'] = fact['Дата'].dt.year
# fact['Неделя'] = fact['Дата'].dt.week

In [ ]:
fact.head()

Добавляем данные в таблицу Факт по географии (исходя из кода грузополучателя) и по товару (исходя из кода товара):

In [11]:
fact_merge = fact.merge(client, on='Код клиента', how='left', validate='m:1')
fact_merge = fact_merge.merge(SKU, on='Код товара', how='left', validate='m:1')
fact_merge = fact_merge[pd.notna(fact_merge['Регион'])]

fact_merge.fillna('', inplace=True)

# fact_merge[pd.isna(fact_merge['Страна'])]
# fact_merge.info()

Группируем данные до необходимых полей (аналог сводной таблицы), сократив данные до областей и продажи до объемов месяца.  
Итоговую размерность таблицы задает сочетание полей "Область - Месяц - товар".  
Это будет итоговая таблица для анализа:

In [ ]:
group_by = ['РФ/Экспорт', 'Канал продаж', 'Регион', 'Страна', 'Месяц', 'Год', 
            'Код товара', 'Наименование товара', 'Категория', 'Бренд']

fact_group = pd.DataFrame({'Объем' : fact_merge.groupby(group_by)['Объем'].sum()}).reset_index()
fact_group[:3]

# Аналитика факта продаж "На первый взгляд". Общие данные

---  
Какая область продавала **наибольшее количество СКЮ в наибольшее количество месяцев** (максимальное количество строк "месяц - СКЮ" среди областей)

In [ ]:
print(f"Колчиество территорий, в которые были продажи: {fact_group['Область РФ/Страна'].nunique()}")
print(f"Территория с наибольшим ассортиментом и наибольшим периодом продаж: {fact_group['Область РФ/Страна'].describe().top}")

---  
Какое СКЮ продаваось с **наибольшей географии в наибольшее количество месяцев** (максимальное количество строк "месяц - область РФ\страна Экспорта" среди СКЮ)

In [ ]:
print(f"Колчиество наименований товара, которые были проданы: {fact_group['Наименование товара'].nunique()}")
print(f"Товар с наибольшей географией и наибольшим периодом продаж: {fact_group['Наименование товара'].describe().top}")
# fact_group['Наименование товара'].describe()

---  
Продажи **Бренда** по убыванию в виде таблицы

In [ ]:
fact_group_ex = pd.DataFrame({'Объем' : fact_group[(fact_group['Категория'] == 'Категория') &
                                                   (fact_group['Бренд'] == 'Бренд')].groupby(['Регион'])['Объем'].sum()}).reset_index()

fact_group_ex['Доля'] = fact_group_ex['Объем'] / fact_group_ex['Объем'].sum()
fact_group_ex.sort_values(by='Объем', ascending=False, inplace=True)
fact_group_ex.reset_index(inplace=True)
fact_group_ex

То же самое на графике

In [ ]:
fact_group_ex = fact_group_ex.loc[:, ['Регион', 'Объем']].head(10)
fact_group_ex['Регион'] = ['\n'.join(wrap(l, 8)) for l in fact_group_ex['Регион']]

fig = plt.figure()
axes = fig.add_axes([0.1, 0.1, 0.7, 0.8]) # left, bottom, width, height (range 0 to 1)
axes.bar(fact_group_ex['Регион'], fact_group_ex['Объем'])
axes.set_xlabel('Регион', weight='bold')
axes.grid(color='gray', alpha=0.5, linestyle='dashed', linewidth=0.5)

for i, j in zip(fact_group_ex['Регион'], fact_group_ex['Объем']):
    axes.annotate(humanize.intcomma(round(j)).replace(',', ' '), xy=(i, j))

---  
Динамика продаж **Категория Бренд** по годам

In [ ]:
fact_group_ex = pd.DataFrame({'Объем' : fact_group[(fact_group['Категория'] == 'Категория') &
                                                   (fact_group['Бренд'] == 'Бренд')].groupby(['Месяц', 'Год'])['Объем'].sum()}).reset_index()

fact_group_ex = pd.crosstab(fact_group_ex['Месяц'], fact_group_ex['Год'], fact_group_ex['Объем'], aggfunc=sum)
fact_group_ex.plot()
# fact_group_ex.plot(kind='area')
# fact_group_ex.plot(kind='bar', rot=0)
# fact_group_ex.plot(kind='pie', subplots=True)

***
Связь между собой различных признаков на примере долей отгрузок в Страну1, Страну2 и Страну3 Категории по брендам в разрезе месяцев и лет  
Исходная таблица данных выглядит следующим образом:

In [ ]:
fact_group_ex = pd.DataFrame({'Объем' : fact_group[(fact_group['Категория'] == 'Категория')
                                                  ].groupby(['Страна', 'Год', 'Месяц', 'SKU: бренд'])['Объем'].sum()}).reset_index()

fact_group_ex = fact_group_ex[(fact_group_ex['Страна'] == 'Страна1') | 
                              (fact_group_ex['Страна'] == 'Страна2') | 
                              (fact_group_ex['Страна'] == 'Страна3')]
fact_group_ex = pd.crosstab(index=[fact_group_ex['Год'], fact_group_ex['Месяц'], fact_group_ex['Бренд']], 
                            columns=fact_group_ex['Страна'], values=fact_group_ex['Объем'], aggfunc=sum).reset_index().fillna(0)
fact_group_ex.head()

Таблица с нормализованными данными (по строчкам сумма равна 100%)

In [ ]:
fact_group_ex = pd.DataFrame({'Объем' : fact_group[(fact_group['Категория'] == 'Категория')
                                                  ].groupby(['Страна', 'Год', 'Месяц', 'Бренд'])['Объем'].sum()}).reset_index()

fact_group_ex = fact_group_ex[(fact_group_ex['Страна'] == 'Страна1') | 
                              (fact_group_ex['Страна'] == 'Страна2') | 
                              (fact_group_ex['Страна'] == 'Страна3')]
fact_group_ex = pd.crosstab(index=[fact_group_ex['Год'], fact_group_ex['Месяц'], fact_group_ex['Бренд']], 
                            columns=fact_group_ex['Страна'], values=fact_group_ex['Объем'], aggfunc=sum, normalize='index').reset_index().fillna(0)
fact_group_ex.head()

График зависимостей полученных данных

In [ ]:
fact_group_ex = pd.DataFrame({'Объем' : fact_group[(fact_group['Категория'] == 'Категория')
                                                  ].groupby(['Страна', 'Год', 'Месяц', 'Бренд'])['Объем'].sum()}).reset_index()

fact_group_ex = fact_group_ex[(fact_group_ex['Страна'] == 'Страна1') | 
                              (fact_group_ex['Страна'] == 'Страна2') | 
                              (fact_group_ex['Страна'] == 'Страна3')]
fact_group_ex = pd.crosstab(index=[fact_group_ex['Год'], fact_group_ex['Месяц'], fact_group_ex['Бренд']], 
                            columns=fact_group_ex['Страна'], values=fact_group_ex['Объем'], aggfunc=sum, normalize='index').reset_index().fillna(0)

fact_group_ex = fact_group_ex.loc[:, ['Бренд', 'Страна1', 'Страна2', 'Страна3']]
fact_group_ex

print(fact_group_ex.describe())
sns.pairplot(fact_group_ex, hue='Бренд',  diag_kind='kde')

---  
Сводная таблица по продажам Категории в разрезе Регионов и брендов

In [ ]:
fact_group_ex = pd.DataFrame({'Объем' : fact_group[(fact_group['Категория'] == 'Категория')
                                                  ].groupby(['Регион', 'Бренд'])['Объем'].sum()}).reset_index()

# Объем в тн
fact_group_ex = pd.crosstab(fact_group_ex['Регион'], fact_group_ex['Бренд'], fact_group_ex['Объем'], 
                            aggfunc=sum, margins=True, margins_name='ИТОГО')
fact_group_ex.fillna('')

# Доля каждого региона внутри бренда
# fact_group_ex = pd.crosstab(fact_group_ex['Регион'], fact_group_ex['Бренд'], fact_group_ex['Объем'], 
#                             aggfunc=sum, normalize='columns')
# fact_group_ex.replace(to_replace=0, value='')